# Evaluation using QCB chemicals (medium)

In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../..')

In [ ]:
import os
from os.path import exists
import copy

import pylab as plt
from loguru import logger
import seaborn as sns
import gymnasium as gym
import pandas as pd
import torch

from vimms.Evaluation import EvaluationData
from vimms.Common import set_log_level_warning, load_obj, save_obj
from vimms.Noise import UniformSpikeNoise
from vimms.Roi import RoiBuilderParams, SmartRoiParams

from vimms_gym.common import METHOD_DQN, evaluate
from vimms_gym.chemicals import generate_chemicals
from vimms_gym.evaluation import run_method
from vimms_gym.common import METHOD_RANDOM, METHOD_TOPN
from vimms_gym.experiments import preset_qcb_medium

from model.QNetwork import get_QNetwork, QNETWORK_CNN
from model.DQN_utils import make_env, masked_epsilon_greedy

# 1. Parameters

Parameters that will be used throughout this notebook. Should be the same as the training ones.

Chemical generation parameters

In [ ]:
env_alpha = 0.25
env_beta = 0.50
extract = True
params, max_peaks = preset_qcb_medium(None, alpha=env_alpha, beta=env_beta, extract_chromatograms=extract)
params, max_peaks

In [ ]:
# n_chemicals = (200, 500)
# mz_range = (100, 600)
# rt_range = (400, 800)
# intensity_range = (1E4, 1E20)

Evaluation parameters

In [ ]:
env_name = 'DDAEnv'
intensity_threshold = 0.5

dqn_in_dir = 'DQN_10.0E6'

# ppo_in_dir = os.path.join('..', '..', 'tune', 'QCB_resimulated_medium')
ppo_in_dir = 'PPO_10.0E6'

methods = [
    METHOD_RANDOM,
    METHOD_TOPN,
]
valid_random = True
n_eval_episodes = 3

In [ ]:
# topN parameters
topN_N = 10
topN_rt_tol = 5
min_ms1_intensity = 5000

# smartroi parameters
alpha = 2
beta = 0.5
ils = 0
smartroi_N = topN_N
smartroi_dew = topN_rt_tol

# weighteddew parameters
t0 = 3
t1 = 15
weighteddew_N = 10

## Generate chemical sets for evaluation

In [ ]:
fname = 'QCB_chems_medium.p'
found = exists(fname)
if found:
    chem_list = load_obj(fname)
    for chems in chem_list:
        print(len(chems))

Generate new chemicals if not found

In [ ]:
params['chemical_creator']

In [ ]:
if not found:
    repeat = 30
    logger.warning('Generating chemicals')
    
    chemical_creator_params = params['chemical_creator']
    chem_list = []
    for i in range(repeat):
        chems = generate_chemicals(chemical_creator_params)
        print(len(chems))
        chem_list.append(chems)
        
    save_obj(chem_list, fname)

In [ ]:
chem_list = chem_list[0:n_eval_episodes]
len(chem_list)

# 2. Evaluation

In [ ]:
set_log_level_warning()

In [ ]:
params

In [ ]:
horizon = 1
out_dir = 'evaluation_10.0E6_horizon_%d_alpha_%.2f_beta_%.2f' % (horizon, env_alpha, env_beta)

In [ ]:
method_eval_results = {}
for method in methods:

    N = 0
    copy_params = copy.deepcopy(params)            
    custom_objects = {
        "learning_rate": 0.0,
        "lr_schedule": lambda _: 0.0,
        "clip_range": lambda _: 0.0,
    }    
    
    model = None
    if method == METHOD_TOPN:
        N = topN_N
        effective_rt_tol = topN_rt_tol
        copy_params['env']['use_dew'] = True
        copy_params['env']['rt_tol'] = effective_rt_tol                        

    banner = 'method = %s max_peaks = %d N = %d rt_tol = %d' % (method, max_peaks, N, 
                                                                                   copy_params['env']['rt_tol'])
    print(banner)
    print()            
            
    episodic_results = run_method(env_name, copy_params, max_peaks, chem_list, method, out_dir, 
                                  N=N, min_ms1_intensity=min_ms1_intensity, model=model,
                                  print_eval=True, print_reward=True, intensity_threshold=intensity_threshold,
                                  mzml_prefix=method, horizon=horizon, valid_random=valid_random)
    eval_results = [er.eval_res for er in episodic_results]
    method_eval_results[method] = eval_results
    print()

#### DQN

In [ ]:
import os

import gymnasium as gym
import pandas as pd
import torch
from vimms.Evaluation import EvaluationData

from vimms_gym.common import METHOD_DQN, evaluate
from model.QNetwork import get_QNetwork, QNETWORK_CNN
from model.DQN_utils import make_env, masked_epsilon_greedy

In [ ]:
device = torch.device("cpu")
device

In [ ]:
model_path = os.path.join(dqn_in_dir, 'DQN.model')
model_path

In [ ]:
env_id = 'DDAEnv'

In [ ]:
envs = gym.vector.SyncVectorEnv([make_env(env_id, 0, max_peaks, params)])
model = get_QNetwork(QNETWORK_CNN, envs, device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

In [ ]:
eval_episodes = n_eval_episodes
eval_episodes, len(chem_list)

In [ ]:
episodic_returns = []
evaluation_results = []
for i in range(len(chem_list)):
    if i >= eval_episodes:
        break

    chems = chem_list[i]

    obs, info = envs.reset(options={'chems': chems})
    done = False
    eval_data = None
    while not done:
        env = envs.envs[0].env
        actions = masked_epsilon_greedy(device, max_peaks, None, obs, model,
                                        deterministic=True)

        next_obs, rewards, terminateds, truncateds, infos = envs.step(actions)
        dones = terminateds

        if 'final_info' in infos:
            final_infos = infos['final_info']
            assert len(final_infos) == 1
            info = final_infos[0]
            episodic_return = info['episode']['r'][0]
            episodic_length = info['episode']['l'][0]

            eval_res = evaluate(eval_data)
            eval_res['invalid_action_count'] = env.invalid_action_count
            eval_res['total_rewards'] = episodic_return
            eval_res['episodic_length'] = episodic_length
            eval_res['num_ms1_scans'] = len(eval_data.controller.scans[1])
            eval_res['num_ms2_scans'] = len(eval_data.controller.scans[2])
            evaluation_results.append(eval_res)

            print(
                f'Episode {i} ({len(chems)} chemicals) return {episodic_return} length {episodic_length}')
            print(eval_res)

            episodic_returns += [episodic_return]
            
            mzml_name = METHOD_DQN
            out_file = '%s_%d.mzML' % (mzml_name, i)
            vimms_env.write_mzML(out_dir, out_file)            

        obs = next_obs
        done = dones[0]

        # store previous results for evaluation before 'done'
        # this needs to be here, because VecEnv is automatically reset when done
        vimms_env = env.env.vimms_env
        eval_data = EvaluationData(vimms_env)

In [ ]:
method_eval_results[METHOD_DQN] = evaluation_results

#### Test classic controllers in ViMMS

In [ ]:
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import TopNController, TopN_SmartRoiController, WeightedDEWController
from vimms.Environment import Environment

Run Top-N Controller

In [ ]:
params

In [ ]:
enable_spike_noise = params['noise']['enable_spike_noise']
ionisation_mode = params['env']['ionisation_mode']
isolation_window = params['env']['isolation_window']
mz_tol = params['env']['mz_tol']
rt_range = params['chemical_creator']['rt_range']

In [ ]:
method = 'TopN_Controller'
print('method = %s' % method)
print()

effective_rt_tol = topN_rt_tol
effective_N = topN_N
eval_results = []
for i in range(len(chem_list)):
    
    spike_noise = None
    if enable_spike_noise:
        noise_params = params['noise']
        noise_density = noise_params['noise_density']
        noise_max_val = noise_params['noise_max_val']
        noise_min_mz = noise_params['mz_range'][0]
        noise_max_mz = noise_params['mz_range'][1]
        spike_noise = UniformSpikeNoise(noise_density, noise_max_val, min_mz=noise_min_mz,
                                        max_mz=noise_max_mz)

    chems = chem_list[i]
    mass_spec = IndependentMassSpectrometer(ionisation_mode, chems, spike_noise=spike_noise)
    controller = TopNController(ionisation_mode, effective_N, isolation_window, mz_tol, effective_rt_tol,
                                min_ms1_intensity)
    env = Environment(mass_spec, controller, rt_range[0], rt_range[1], progress_bar=False, out_dir=out_dir,
                      out_file='%s_%d.mzML' % (method, i), save_eval=True)
    env.run()
    
    eval_res = evaluate(env, intensity_threshold)
    # eval_res['total_rewards'] = 0
    eval_results.append(eval_res)
    print('Episode %d finished' % i)
    print(eval_res)
    print()
    
method_eval_results[method] = eval_results

Run SmartROI Controller

In [ ]:
smartroi_N, smartroi_dew, alpha, beta, ils

In [ ]:
method = 'SmartROI_Controller'
print('method = %s' % method)
print()

eval_results = []
for i in range(len(chem_list)):
    
    spike_noise = None
    if enable_spike_noise:
        noise_params = params['noise']
        noise_density = noise_params['noise_density']
        noise_max_val = noise_params['noise_max_val']
        noise_min_mz = noise_params['mz_range'][0]
        noise_max_mz = noise_params['mz_range'][1]
        spike_noise = UniformSpikeNoise(noise_density, noise_max_val, min_mz=noise_min_mz,
                                        max_mz=noise_max_mz)

    chems = chem_list[i]
    mass_spec = IndependentMassSpectrometer(ionisation_mode, chems, spike_noise=spike_noise)
    
    roi_params = RoiBuilderParams(min_roi_intensity=0, min_roi_length=0)    
    smartroi_params = SmartRoiParams(intensity_increase_factor=alpha, drop_perc=beta/100.0, dew=smartroi_dew, initial_length_seconds=ils)
    controller = TopN_SmartRoiController(ionisation_mode, isolation_window, smartroi_N, mz_tol, smartroi_dew,
                                min_ms1_intensity, roi_params, smartroi_params)
    env = Environment(mass_spec, controller, rt_range[0], rt_range[1], progress_bar=False, out_dir=out_dir,
                      out_file='%s_%d.mzML' % (method, i), save_eval=True)
    env.run()
    
    eval_res = evaluate(env, intensity_threshold)
    # eval_res['total_rewards'] = 0
    eval_results.append(eval_res)
    print('Episode %d finished' % i)
    print(eval_res)
    print()    
    
method_eval_results[method] = eval_results

Run WeightedDEW Controller

In [ ]:
method = 'WeightedDEW_Controller'
print('method = %s' % method)
print()

eval_results = []
for i in range(len(chem_list)):
    
    spike_noise = None
    if enable_spike_noise:
        noise_params = params['noise']
        noise_density = noise_params['noise_density']
        noise_max_val = noise_params['noise_max_val']
        noise_min_mz = noise_params['mz_range'][0]
        noise_max_mz = noise_params['mz_range'][1]
        spike_noise = UniformSpikeNoise(noise_density, noise_max_val, min_mz=noise_min_mz,
                                        max_mz=noise_max_mz)

    chems = chem_list[i]
    mass_spec = IndependentMassSpectrometer(ionisation_mode, chems, spike_noise=spike_noise)
    
    controller = WeightedDEWController(ionisation_mode, weighteddew_N, isolation_window, mz_tol, t1,
                                min_ms1_intensity, exclusion_t_0=t0)
    env = Environment(mass_spec, controller, rt_range[0], rt_range[1], progress_bar=False, out_dir=out_dir,
                      out_file='%s_%d.mzML' % (method, i), save_eval=True)
    env.run()
    
    eval_res = evaluate(env, intensity_threshold)
    # eval_res['total_rewards'] = 0
    eval_results.append(eval_res)
    print('Episode %d finished' % i)
    print(eval_res)
    print()    
    
method_eval_results[method] = eval_results

#### Plotting

Flatten data into dataframe

In [ ]:
data = []
for method in method_eval_results:
    eval_results = method_eval_results[method]
    for eval_res in eval_results:
        try:
            total_rewards = float(eval_res['total_rewards'])       
        except KeyError:
            total_rewards = 0.0

        try:
            invalid_action_count = float(eval_res['invalid_action_count'])       
        except KeyError:
            invalid_action_count = 0.0

        row = (
            method, 
            total_rewards, 
            invalid_action_count,
            float(eval_res['coverage_prop']), 
            float(eval_res['intensity_prop']), 
            float(eval_res['ms1ms2_ratio']), 
            float(eval_res['efficiency']),
            float(eval_res['precision']),
            float(eval_res['recall']),
            float(eval_res['f1']),            
        )
        data.append(row)
        
df = pd.DataFrame(data, columns=['method', 'total_rewards', 'invalid_action_count', 
    'coverage_prop', 'intensity_prop', 'ms1/ms2_ratio', 'efficiency', 
    'precision', 'recall', 'f1'])
# df.set_index('method', inplace=True)
df.head()

In [ ]:
sns.set_context("poster")

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='method', y='total_rewards')
plt.xticks(rotation=90)
plt.title('Total Rewards')

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='method', y='invalid_action_count')
plt.xticks(rotation=90)
plt.title('Invalid Action Count')

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='method', y='coverage_prop')
plt.xticks(rotation=90)
plt.title('Coverage Proportion')

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='method', y='intensity_prop')
plt.xticks(rotation=90)
plt.title('Intensity Proportion')

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='method', y='ms1/ms2_ratio')
plt.xticks(rotation=90)
plt.title('MS1/MS2 Ratio')

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='method', y='efficiency')
plt.xticks(rotation=90)
plt.title('Efficiency')

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='method', y='precision')
plt.xticks(rotation=90)
plt.title('Precision')

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='method', y='recall')
plt.xticks(rotation=90)
plt.title('Recall')

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='method', y='f1')
plt.xticks(rotation=90)
plt.title('F1')

In [ ]:
df.to_pickle('evaluation_10.0E6_horizon_%d_alpha_%.2f_beta_%.2f.p' % (horizon, env_alpha, env_beta))